In [ ]:
import sys
sys.path.insert(0, '/Users/base/Desktop/G-AI') # Adjust the path 

In [ ]:
from langchain.tools import tool
from RAG_1.RAG.tools import search_internet

@tool
def search(query: str) -> str:
    """Search for information on internet."""
    result = search_internet(query=query)
    return f"Results for: {query}:\n\n {result}"

In [ ]:
from langchain_groq import ChatGroq
from langchain.agents import create_agent

# Groq model
GROQ_MODEL = "llama-3.1-8b-instant"

# Initialize Groq-backed LLM for LangChain
model = ChatGroq(
    model=GROQ_MODEL,
    temperature=0.1,
    max_tokens=1000,
    timeout=30
)

# Build agent with Groq model + LangChain tools
agent = create_agent(model, tools=[search])


In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in San Francisco?"}]}
)
result

In [ ]:

from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse


basic_model = ChatGroq(
    model=GROQ_MODEL,
    temperature=0.1,
    max_tokens=1000,
    timeout=30
)
advanced_model = ChatGroq(
    model=GROQ_MODEL,
    temperature=0.1,
    max_tokens=1000,
    timeout=30
)

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))

agent = create_agent(
    model=basic_model,  # Default model
    tools=[],
    middleware=[dynamic_model_selection]
)

In [ ]:
"testing middleware"

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_groq import ChatGroq


def read_email_tool(email_id: str) -> str:
    """Mock function to read an email by its ID."""
    return f"Email content for ID: {email_id}"

def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """Mock function to send an email."""
    return f"Email sent to {recipient} with subject '{subject}'"


# Groq model
GROQ_MODEL = "llama-3.1-8b-instant"

# Initialize Groq-backed LLM for LangChain
model = ChatGroq(
    model=GROQ_MODEL,
    temperature=0.1,
    max_tokens=1000,
    timeout=30
)
agent = create_agent(
    model=model,
    tools=[read_email_tool, send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                },
                "read_email_tool": False,
            }
        ),
    ],
)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

agent.invoke({
    "messages": [
        {"role": "user", "content": "Please read the email with ID 12345 and send a reply.[mock test call the avl tools]"}
    ]
}, config=config)


In [1]:
# Human in the loop

import os

for k in ["LANGCHAIN_TRACING_V2", "LANGCHAIN_API_KEY", "LANGCHAIN_PROJECT", "LANGCHAIN_ENDPOINT"]:
    print(k, "=", os.getenv(k))


LANGCHAIN_TRACING_V2 = true
LANGCHAIN_API_KEY = add_new_key
LANGCHAIN_PROJECT = FirstLangChainProject
LANGCHAIN_ENDPOINT = None


In [2]:
from langsmith import Client

client = Client()

In [7]:
from langsmith import traceable

@traceable(name="notebook-test-trace")
def foo(x):
    return x * 2

print(foo(21))



42


In [3]:
print(client.list_projects())

<generator object Client.list_projects at 0x10d9b0970>


In [4]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware, LLMToolEmulator
from langgraph.checkpoint.memory import InMemorySaver 
from langchain_groq import ChatGroq
from langchain.tools import tool


@tool
def write_file_tool(file_path: str, content: str) -> str:
    """Mock function to write content to a file."""
    return f"Written to {file_path}"
@tool
def execute_sql_tool(query: str) -> str:
    """Mock function to execute an SQL query."""
    return f"Executed SQL query: {query}"
@tool
def read_data_tool(data_id: str) -> str:
    """Mock function to read data by its ID."""
    return f"Data for ID: {data_id}"


# Groq model
GROQ_MODEL = "llama-3.1-8b-instant"

# Initialize Groq-backed LLM for LangChain
model = ChatGroq(
    model=GROQ_MODEL,
    temperature=0.1,
    max_tokens=1000,
    timeout=30
)

agent = create_agent(
    model=model,
    tools=[write_file_tool, execute_sql_tool, read_data_tool],
    middleware=[
        LLMToolEmulator(model=model,tools=["write_file_tool","execute_sql_tool","read_data_tool"]),  # Emulate tool calls for testing
        HumanInTheLoopMiddleware( 
            interrupt_on={
                "write_file_tool": True,  # All decisions (approve, edit, reject) allowed
                "execute_sql_tool": {"allowed_decisions": ["approve", "reject"]},  # No editing allowed
                # Safe operation, no approval needed
                "read_data_tool": False,
            },
            # Prefix for interrupt messages - combined with tool name and args to form the full message
            # e.g., "Tool execution pending approval: execute_sql with query='DELETE FROM...'"
            # Individual tools can override this by specifying a "description" in their interrupt config
            description_prefix="Tool execution pending approval",
        ),
    ],
    # Human-in-the-loop requires checkpointing to handle interrupts.
    # In production, use a persistent checkpointer like AsyncPostgresSaver.
    checkpointer=InMemorySaver(),  
)

/Users/base/Desktop/G-AI/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langgraph.types import Command

# Human-in-the-loop leverages LangGraph's persistence layer.
# You must provide a thread ID to associate the execution with a conversation thread,
# so the conversation can be paused and resumed (as is needed for human review).
config = {"configurable": {"thread_id": "some_id"}} 
# Run the graph until the interrupt is hit.
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Delete old records from the database",
            }
        ]
    },
    config=config 
)

# The interrupt contains the full HITL request with action_requests and review_configs
print(result['__interrupt__'])
# > [
# >    Interrupt(
# >       value={
# >          'action_requests': [
# >             {
# >                'name': 'execute_sql',
# >                'arguments': {'query': 'DELETE FROM records WHERE created_at < NOW() - INTERVAL \'30 days\';'},
# >                'description': 'Tool execution pending approval\n\nTool: execute_sql\nArgs: {...}'
# >             }
# >          ],
# >          'review_configs': [
# >             {
# >                'action_name': 'execute_sql',
# >                'allowed_decisions': ['approve', 'reject']
# >             }
# >          ]
# >       }
# >    )
# > ]


# Resume with approval decision
agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "edit", "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

[Interrupt(value={'action_requests': [{'name': 'execute_sql_tool', 'args': {'query': 'DELETE FROM records WHERE created_at < DATE_SUB(CURRENT_DATE, INTERVAL 30 DAY)'}, 'description': "Tool execution pending approval\n\nTool: execute_sql_tool\nArgs: {'query': 'DELETE FROM records WHERE created_at < DATE_SUB(CURRENT_DATE, INTERVAL 30 DAY)'}"}], 'review_configs': [{'action_name': 'execute_sql_tool', 'allowed_decisions': ['approve', 'reject']}]}, id='965387824a58f3dcd3adf30581e27d19')]


{'messages': [HumanMessage(content='Delete old records from the database', additional_kwargs={}, response_metadata={}, id='e7af938c-d365-4419-b094-c1667f9db62f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'bd3g9qnq9', 'function': {'arguments': '{"query":"DELETE FROM records WHERE created_at \\u003c DATE_SUB(CURRENT_DATE, INTERVAL 30 DAY)"}', 'name': 'execute_sql_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 348, 'total_tokens': 381, 'completion_time': 0.047241473, 'prompt_time': 0.041788036, 'queue_time': 0.060652954, 'total_time': 0.089029509}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--628acd74-eed4-40c4-b580-df7202c68c18-0', tool_calls=[{'name': 'execute_sql_tool', 'args': {'query': 'DELETE FROM records WHERE created_at < DATE_SUB(CURRENT_DATE, INTERV

In [ ]:
# Resume with approval decision
agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "edit", "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)